In [3]:
from langchain.llms import Ollama 

from PyPDF2 import PdfReader

from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory


In [4]:

llm = Ollama(model='mistral', temperature=0.7)
def get_conversation_chain(file):
    text = ''
    reader = PdfReader(file)
    for page in reader.pages:
        text += page.extract_text()

    text_splitter = CharacterTextSplitter(
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap = 200,
    length_function = len
    )

    chunks = text_splitter.split_text(text)


    embeddings = HuggingFaceEmbeddings()

    vectorstore = FAISS.from_texts(texts = chunks , embedding = embeddings)

    memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

    conversation_chain = ConversationalRetrievalChain.from_llm(
        llm = llm,
        retriever = vectorstore.as_retriever(search_kwargs={"k": 2}),
        memory = memory
    )

In [6]:
while True:
  query = input('Enter your query regarding the pdf : ')

  if query.lower() == 'exit':
    break


  result = conversation_chain({"question": query })

  print(result['answer'])
  


I'm sorry, but the provided context does not contain any information about machine learning. Could you please provide more context or clarify your question?


KeyboardInterrupt: 

In [13]:
from flask import Flask, request, jsonify
from flask_cors import CORS
from werkzeug.utils import secure_filename
import os
import traceback

app = Flask(__name__)
CORS(app)



UPLOAD_FOLDER = './uploads'

app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER

def allowed_file(filename):
    return '.' in filename and filename.rsplit('.', 1)[1].lower() == 'pdf'


@app.route('/upload', methods=['POST'])
def upload_file():

    if 'file' not in request.files:
        return jsonify({'error': 'No file part'})

    file = request.files['file']

    if file.filename == '':
        return jsonify({'error': 'No selected file'})

    if file and allowed_file(file.filename):
        filename = secure_filename(file.filename)
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        try:
            file.save(filepath)
        except Exception as e:
            return jsonify({'error': f'Error saving file: {str(e)}'})
        
        global conversation_chain
        conversation_chain = get_conversation_chain(file)
        return jsonify({'message': 'PDF successfully read! You can now ask me any question regarding it.'})

    return jsonify({'error': 'Invalid file'})

@app.route('/message', methods=['POST'])
def get_message():
    data = request.get_json()

    if 'message' not in data:
        return jsonify({'error': 'data not found'})
    
    message = data['message']

    result = conversation_chain({"question": message })

    return jsonify({'response':result['answer']})



    

if __name__ == '__main__':
    app.run(debug=True)


 * Serving Flask app '__main__'
 * Debug mode: on


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
 * Restarting with stat


SystemExit: 1